In [ ]:
from __future__ import print_function
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from utils import pgd_, fgsm_, gradient_information, adversarial_accuracy
from Nets import MNIST_Net
%load_ext autoreload
%autoreload 2
%aimport utils, Nets

## Train a NeuralNet to run experiments on

In [4]:
device = torch.device("cuda")
batch_size = 64
test_batch_size = 1000
epochs = 14
log_interval = 100

model = MNIST_Net(device=device, log_interval=log_interval, batch_size=batch_size, test_batch_size=test_batch_size).to(device)
model.train_on_data(epochs)

## Attack

In [11]:
adversarial_dataset = torch.utils.data.Subset(test_dataset, [i for i in range(1000)])
adversarial_loader = torch.utils.data.DataLoader(adversarial_dataset, batch_size=2, num_workers=2, shuffle=False)
adversarial_accuracy(model, adversarial_loader)

0 / 1000
200 / 1000
400 / 1000
600 / 1000
800 / 1000
68.5


## Gradient masking metric
Check alignment of gradient at adv point with (adv point - original point)

In [24]:
n_examples = 1000
x = torch.cat([train_dataset[i][0].unsqueeze(0) for i in range(n_examples)]).to(device)
y = torch.LongTensor([train_dataset[i][1] for i in range(n_examples)]).to(device)

In [28]:
[(i, np.nanmean(gradient_information(model, x, y, iters=i, clip_min=normalized_min, clip_max=normalized_max, device=device).detach().cpu().numpy())) for i in [j*10 for j in range(1, 52, 5)]]

[(10, 0.20104747),
 (60, 0.5395799),
 (110, 0.55219793),
 (160, 0.5526937),
 (210, 0.55167156),
 (260, 0.55157655),
 (310, 0.5525947),
 (360, 0.5521684),
 (410, 0.55295753),
 (460, 0.5509018),
 (510, 0.5505047)]

## Adversarial Training

In [48]:
def adv_train(model, device, train_loader, optimizer, epochs):
    model.train()
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs + 1):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            adv_data = pgd_(model, data, target, 0.1, 0.5, iters=7, targeted=False, device=device, clip_min=normalized_min, clip_max=normalized_max)
            optimizer.zero_grad()
            output = model(adv_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

In [55]:
undefended_model = type(model)().to(device)
undefended_model.load_state_dict(model.state_dict())
adv_train(model, device, train_loader, optimizer, 4)

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.735852
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.935562
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.539442
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.542072
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.375298
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.234662
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.547802
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.331915
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.199902
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.282771
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.450014
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.228553
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.216324
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.487287
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.375047
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.198464
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.329873
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.318543
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.161249
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.292225
Train Epoch:

In [56]:
adversarial_accuracy(model, adversarial_loader)

0 / 1000
200 / 1000
400 / 1000
600 / 1000
800 / 1000
94.19999999999999


## Black Box Attack

In [51]:
def black_box_adversarial_accuracy(model, undefended_model, dataset_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(dataset_loader):
        data, target = data.to(device), target.to(device)
        adv = pgd_(undefended_model, data, target, 0.1, 0.5, iters=7, targeted=False, device=device, clip_min=normalized_min, clip_max=normalized_max)
        output = model(adv)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        if (batch_idx % 100 == 0):
            print('{} / {}'.format(batch_idx * dataset_loader.batch_size, len(dataset_loader.dataset)))
    print ((correct/len(dataset_loader.dataset) * 100))

In [57]:
black_box_adversarial_accuracy(model, undefended_model, adversarial_loader)

0 / 1000
200 / 1000
400 / 1000
600 / 1000
800 / 1000
94.0
